## Affectus analysis


Install libraries


In [94]:
! pip install --upgrade setuptools wheel
! pip install kaggle
! pip install numpy
! pip install pandas
! pip install nltk
! pip install scikit-learn
! pip install nyoka


APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL=true
APP_DEBUG=true
APP_ENV=local
APP_HOST=127.0.0.1
APP_KEY=mTO7A4wTQC2NRz76UtSB2vmosVBjMv2g
APP_NAME='Affectus Analysis'
APP_PORT=3000
APP_URL=http://localhost
CLICOLOR=1
CLICOLOR_FORCE=1
COMMAND_MODE=unix2003
DB_CONNECTION=mysql
DB_DATABASE=affectus-analysis
DB_HOST=127.0.0.1
DB_PASSWORD=''
DB_PORT=3306
DB_USERNAME=root
ELECTRON_NO_ATTACH_CONSOLE=1
ELECTRON_RUN_AS_NODE=1
FORCE_COLOR=1
GIT_PAGER=cat
GO111MODULE=on
GOMODCACHE=/Users/arfanxn/Developments/Golang/pkg/mod
GOPATH=/Users/arfanxn/Developments/Golang/
GOPROXY=https://proxy.golang.org,direct
GOROOT=/opt/homebrew/opt/go/libexec
GRPC_HOST=''
GRPC_PORT=''
HOME=/Users/arfanxn
HOMEBREW_CELLAR=/opt/homebrew/Cellar
HOMEBREW_PREFIX=/opt/homebrew
HOMEBREW_REPOSITORY=/opt/homebrew
INFOPATH=/opt/homebrew/share/info:
JWT_SECRET=6kKEQ1ZXYlCo18odfg8m7NXZDyH50f8T
KMP_DUPLICATE_LIB_OK=True
KMP_INIT_AT_FORK=FALSE
LC_CTYPE=UTF-8
LESS=-R
LOGNAME=arfanxn
LOG_CHANNEL=stack
LOG_LEVEL=debug
LSCOLORS=Gxfxcxd

Import the dependencies


In [102]:
import numpy as np
import pandas as pd
import re 
import pickle 
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from nyoka import skl_to_pmml


nltk.download('stopwords')  # stopwords.words('english') # print to see what is stopwords.words('english')
stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arfanxn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Vars / Configs / Envs


In [99]:
# ! kaggle config set username <your_kaggle_username>
# ! kaggle config set key <your_kaggle_key>

kaggle_dataset_name = 'kazanova/sentiment140'

dataset_path = './datasets'
dataset_zip_filename = 'sentiment140.zip' 
dataset_csv_filename =  'training.1600000.processed.noemoticon.csv'
dataset_colum_names = ['target', 'id', 'date', 'flag', 'user', 'text'] 

model_path = './models'
model_filename_without_ext = 'affectues-analysis'


Download dataset


In [31]:
# download dataset from kaggle api
! kaggle datasets download -d {kaggle_dataset_name} -p {dataset_path} 

APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL=true
APP_DEBUG=true
APP_ENV=local
APP_HOST=127.0.0.1
APP_KEY=mTO7A4wTQC2NRz76UtSB2vmosVBjMv2g
APP_NAME='Affectus Analysis'
APP_PORT=3000
APP_URL=http://localhost
CLICOLOR=1
CLICOLOR_FORCE=1
COMMAND_MODE=unix2003
DB_CONNECTION=mysql
DB_DATABASE=affectus-analysis
DB_HOST=127.0.0.1
DB_PASSWORD=''
DB_PORT=3306
DB_USERNAME=root
ELECTRON_NO_ATTACH_CONSOLE=1
ELECTRON_RUN_AS_NODE=1
FORCE_COLOR=1
GIT_PAGER=cat
GO111MODULE=on
GOMODCACHE=/Users/arfanxn/Developments/Golang/pkg/mod
GOPATH=/Users/arfanxn/Developments/Golang/
GOPROXY=https://proxy.golang.org,direct
GOROOT=/opt/homebrew/opt/go/libexec
GRPC_HOST=''
GRPC_PORT=''
HOME=/Users/arfanxn
HOMEBREW_CELLAR=/opt/homebrew/Cellar
HOMEBREW_PREFIX=/opt/homebrew
HOMEBREW_REPOSITORY=/opt/homebrew
INFOPATH=/opt/homebrew/share/info:
JWT_SECRET=6kKEQ1ZXYlCo18odfg8m7NXZDyH50f8T
KMP_DUPLICATE_LIB_OK=True
KMP_INIT_AT_FORK=FALSE
LC_CTYPE=UTF-8
LESS=-R
LOGNAME=arfanxn
LOG_CHANNEL=stack
LOG_LEVEL=debug
LSCOLORS=Gxfxcxd

In [32]:
# extract the downloaded dataset(s) then remove the zip file(s)
from zipfile import ZipFile
import os

with ZipFile(dataset_path + '/' + dataset_zip_filename, 'r') as zip:
    zip.extractall(dataset_path)

os.remove(dataset_path + '/' + dataset_zip_filename) 

Data processing


In [33]:
# Load the data from csv to pandas DataFrame
df = pd.read_csv(dataset_path + '/' +dataset_csv_filename, names=dataset_colum_names, encoding='ISO-8859-1') 

In [34]:
# check the number of rows and columns
df.shape

(1600000, 6)

In [35]:
# Print df
print(df.head(), df.tail())

   target          id                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....            target          id                          date      flag  \
1599995       4  2193601966  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599996       4  2193601969  Tue Ju

In [36]:
# count the number of missing values in the dataset 
# (in this case the values are present in the dataset so we don't need to do further processing)
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

Convert the target "4" to "1"


In [37]:
df.replace({'target':{4:1}}, inplace=True) 

In [38]:
# check the distribution of target column
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 = Negative, 1 = Positive


**Stemming**

\n is a process of reducing a word to its Root word
\n example: actor, actress, acting = act


In [39]:
porter_stemmer = PorterStemmer() 

def stem_text(text : str):
    text = re.sub('[^a-zA-Z]',' ', text) 
    text = text.lower() 
    words = text.split() 
    words = [porter_stemmer.stem(word) for word in words if not word in stopwords.words('english')] 
    text = ' '.join(words) 
    return text 

In [40]:
df['stemmed_text'] = df['text'].apply(stem_text)  #!!!  this will take 30-60 minutes to complete the operation !!!#

In [103]:
# Test stem individual text
stem_text("BBRI shares are starting to improve after last week's free fall")

'bbri share start improv last week free fall'

In [41]:
df.head()

,target,id,date,flag,user,text,stemmed_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [42]:
# separate the stemmed_text and label 
X = df['stemmed_text'].values.astype('str')
Y = df['target'].values

In [43]:
print('stemmed_text: ', X, '\ntarget: ', Y)

stemmed_text:  ['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h'] 
target:  [0 0 0 ... 1 1 1]


Split the data for training and testing purposes


In [106]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=7)
print(X_test)

['oh forgot import birthday today mrsrestrictor redact year old today'
 'tire yawn best work'
 'wonder time tonight stinkin awesom famili even chelsey bo alright' ...
 'eat sushi' 'hotnsaucey day good friend' 'yay oral final exam']


In [45]:
print('X.shape: ', X.shape, 
      '\nX_train.shape: ', X_train.shape, 
      '\nX_test.shape: ', X_test.shape, 
      '\n\nY.shape: ', Y.shape, 
      '\nY_train.shape:', Y_train.shape, 
      '\nY_test.shape:', Y_test.shape)

X.shape:  (1600000,) 
X_train.shape:  (1280000,) 
X_test.shape:  (320000,) 

Y.shape:  (1600000,) 
Y_train.shape: (1280000,) 
Y_test.shape: (320000,)


In [46]:
# convert textual to numerical representation

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [47]:
print(X_train)

  (0, 177827)	0.21427584629824487
  (0, 78681)	0.22576199333749816
  (0, 31110)	0.20773704112163793
  (0, 279423)	0.3428184401343486
  (0, 138742)	0.24513704938905198
  (0, 170258)	0.2762802934137004
  (0, 262111)	0.3555114768058745
  (0, 86882)	0.30159464794998875
  (0, 237192)	0.26537306011695083
  (0, 49383)	0.2981972422694594
  (0, 453293)	0.26730987929913863
  (0, 300402)	0.28164691480372833
  (0, 155372)	0.28025612403710826
  (1, 177827)	0.1884727983145768
  (1, 78681)	0.19857578617690264
  (1, 350909)	0.5440254661314423
  (1, 145799)	0.1556199416062376
  (1, 453295)	0.4089354835572553
  (1, 305640)	0.3826922105576465
  (1, 70180)	0.30992185225350954
  (1, 299033)	0.1972545806518825
  (1, 230543)	0.24061578831492064
  (1, 322661)	0.31356968603824703
  (2, 409653)	1.0
  (3, 151653)	0.1254797626373641
  :	:
  (1279994, 375510)	0.249636768338832
  (1279994, 59836)	0.35560296474396647
  (1279994, 124625)	0.23912813000249217
  (1279994, 46514)	0.34233118338329266
  (1279994, 393508)	0

In [48]:
print(X_test)

  (0, 42799)	0.23259338082271522
  (0, 136191)	0.25595893278772636
  (0, 178709)	0.30763458557423734
  (0, 277123)	0.5498437106140087
  (0, 299033)	0.18013019468696764
  (0, 300402)	0.2262246261280556
  (0, 335523)	0.5019062696098175
  (0, 411033)	0.3190359682563617
  (0, 453293)	0.2147088227361715
  (1, 39459)	0.44467946585255413
  (1, 409653)	0.4414973538142055
  (1, 445317)	0.314363397907563
  (1, 452765)	0.7131030172984495
  (2, 12453)	0.3140750440261823
  (2, 28897)	0.22575268477323093
  (2, 46494)	0.3964047789290299
  (2, 68871)	0.4881406163788745
  (2, 124343)	0.21757689112411835
  (2, 127912)	0.2603631559671138
  (2, 384351)	0.4489961539402112
  (2, 408641)	0.17446103046604197
  (2, 412343)	0.21181427109967296
  (2, 444429)	0.2508912564017652
  (3, 25718)	0.4365895598107333
  (3, 127762)	0.4256000962062077
  :	:
  (319994, 18696)	0.6916169953040396
  (319994, 102965)	0.499039675674248
  (319994, 131790)	0.444678567420096
  (319994, 349338)	0.2736536233759103
  (319995, 25906)	0

Train the logistic regression model


In [49]:
# Directly train with sklearn LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model evaluation accuracy score


In [50]:
# accuracy score on the training data
X_train_pred = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_pred)
print("training_data_accuracy: " + str(training_data_accuracy))

training_data_accuracy: 0.80483046875


In [51]:
# accuracy score on the testing data
X_test_pred = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_pred)
print("testing_data_accuracy: " + str(testing_data_accuracy))

testing_data_accuracy: 0.778946875


Save the trained model with pickle and pmml


In [117]:
# save with pickle
pickle.dump(model, open(model_path + '/' + model_filename_without_ext + '.sav', 'wb'))

# save additional required parameters
pickle.dump(vectorizer, open(model_path + "/" + "tfidf_vectorizer.pickle", "wb"))

In [100]:
# save with pmml

pipeline_model = Pipeline([("logistic_regression", model)])
pipeline_model.fit(X_train,Y_train)
pmml_file_destination = model_path + '/' + model_filename_without_ext + '.pmml'
skl_to_pmml(pipeline=pipeline_model,col_names=dataset_colum_names,target_name="target",pmml_f_name=pmml_file_destination)

Use the saved model for future prediction


In [116]:
loaded_model = pickle.load(open(model_path + '/' + model_filename_without_ext + ".sav", 'rb'))

X_new = vectorizer.transform([stem_text("BBRI shares are starting to improve after last week's free fall")])[0]

print(X_new)
# print('X_new: ' + str(Y_test[200]))

X_new_predicted = loaded_model.predict(X_new)
print('X_new_predicted: ' + str(X_new_predicted[0]))
print('X_new_predicted is a ' + ('Positive' if X_new_predicted[0] == 1 else 'Negative'))

  (0, 127762)	0.39869951870670917
  (0, 137941)	0.37336755318090165
  (0, 178770)	0.5095817584587741
  (0, 227852)	0.28285721318738155
  (0, 361987)	0.4183050083609117
  (0, 381680)	0.3126657559490902
  (0, 437943)	0.298688194352637
X_new_predicted: 1
X_new_predicted is a Positive
